In [1]:
# Allow reload of objects
%load_ext autoreload
%autoreload
from pelenet.utils import Utils
from pelenet.experiments.readout import ReadoutExperiment

# Official modules
import numpy as np
import nxsdk.api.n2a as nx
import matplotlib
import matplotlib.pyplot as plt
from time import time

In [2]:
# Initilizes the experiment and utils
utils = Utils.instance()
exp = ReadoutExperiment()

In [3]:
# Build network
exp.build()

# Get spectral radius
utils.getSpectralRadius(exp.net.initialWeights)

/home/seq/.local/share/virtualenvs/seq-r0_WvjRo/lib/python3.5/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


1.25

In [ ]:
# Run network
exp.run()

INFO:DRV:  Connecting to 127.0.0.1:39839
INFO:DRV:      Host server up..............Done 0.05s
INFO:DRV:      Encoding axons/synapses.....Done 15.33s
INFO:DRV:      Compiling Embedded snips....Done 0.17s
INFO:HST:  Using Kapoho Bay serial number 308
INFO:HST:  Args chip=0 cpu=0 /home/seq/.local/share/virtualenvs/seq-r0_WvjRo/lib/python3.5/site-packages/nxsdk/driver/compilers/../../../temp/1575902233.2327538/launcher_chip0_lmt0.bin --chips=2 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=1 cpu=0 /home/seq/.local/share/virtualenvs/seq-r0_WvjRo/lib/python3.5/site-packages/nxsdk/driver/../bin/x86/kb/init_bs.bin --chips=2 --remote-relay=0 --epoch=0 
INFO:HST:  Lakemont_driver...
INFO:DRV:      Booting up..................Done 1.26s
INFO:DRV:      Encoding probes.............Done 0.01s
INFO:DRV:      Configuring registers.......Done 17.44s
INFO:DRV:      Transferring probes.........Done 0.04s
INFO:DRV:      Transferring spikes.........Done 0.05s
INFO:HST:  chip=1 cpu=0 halted, status=0x0


In [ ]:
exp.net.plot.landscape()

In [ ]:
exp.net.plot.reservoirSpikeTrain()

In [ ]:
exp.net.plot.reservoirSpikeTrain(fr=0, to=1750)

In [ ]:
exp.p.stopRelaxation

In [ ]:
#exp.net.exVoltageProbes[0].data[0,:100]

In [ ]:
p = exp.net.exVoltageProbes[0].plot()

In [ ]:
p = exp.net.exCurrentProbes[0].plot()

In [ ]:
voltageExSums = []
for j in range(400,650):
    voltageSum =  np.sum([ exp.net.exVoltageProbes[i].data[:,j] for i in range(len(exp.net.exVoltageProbes)) ])
    voltageExSums.append(voltageSum)
p = plt.plot(voltageExSums)

In [ ]:
print(voltageExSums[-1])

In [ ]:
p = exp.net.inVoltageProbes[0].plot()

In [ ]:
p = exp.net.inCurrentProbes[0].plot()

In [ ]:
voltageSums = []
for j in range(400,500):
    voltageSum =  np.sum([ exp.net.inVoltageProbes[i].data[:,j] for i in range(len(exp.net.inVoltageProbes)) ])
    voltageSums.append(voltageSum)
p = plt.plot(voltageSums)

In [ ]:
print(voltageSums[-1])

In [ ]:
offset = exp.p.cueSteps + exp.p.cueRelaxation
data = np.array([ exp.net.exSpikeTrains[:,offset+i*exp.p.trialSteps:offset+i*exp.p.trialSteps+exp.p.movementSteps] for i in range(exp.p.trials)])
shp = data.shape
shp

In [ ]:
np.sum(np.abs(data[0] - data[1]))/(shp[1]*shp[2])

In [ ]:
# TODO Calculate for every binned vector
err = np.sum(np.abs(data[0] - data[1]), axis=0)/(shp[1])
p = plt.plot(err)

In [ ]:
binSize = 10
(params, ye, performance) = utils.trainOLS(data[:5], data[5], exp.targetFunction, filter='bins', binSize=binSize)

plt.title(performance.cor)
plt.plot(ye)
plt.plot(np.array([np.mean(exp.targetFunction[i:i+binSize]) for i in range(0,len(exp.targetFunction),binSize)]))
p = plt.show()